# Chapter 06: Example Application

**Declarative coding:** Focuses on describing what you want to achieve without specifying how to do it. The system handles the implementation details (e.g., SQL). \

**Imperative coding:** Focuses on describing how to achieve a result through explicit instructions or steps, manipulating the program's state (e.g., C, Java).

**Differences:**
- _Declarative:_ Focuses on the what
- _Imperative:_ Focuses on the how

First Example: 

In [1]:
class Car:
    def __init__(self, name):
        self.make = name

cars = [Car('Jaguar'), Car('Volvo')]

In [2]:
# imperative
makes = []
for i in range(len(cars)):
    makes.append(cars[i].make)
makes

['Jaguar', 'Volvo']

In [3]:
# declarative
makes = list(map(lambda car: car.make, cars))
makes

['Jaguar', 'Volvo']

Second Example:

In [4]:
users = {'test': 'pass123'}
toUser = lambda form: form['username'] if form['username'] in users and users[form['username']]==form['password'] else None 
logIn = lambda x: print(f'User {x} has logged in!') if x in users else print('User or Password Incorrect')

In [5]:
# imperative
def authenticate(form):
  user = toUser(form)
  return logIn(user)

# because there is still an encoded step-by-step evaluation within in it
authenticate({'username': 'test', 'password': 'pass123'})

User test has logged in!


In [6]:
import utils

# declarative
authenticate = utils.compose(logIn, toUser)

authenticate({'username': 'test', 'password': 'pass123'})

User test has logged in!


### Let's explore the Flickr of Functional Programming

In [7]:
host = 'api.flickr.com'
path = '/services/feeds/photos_public.gne'

query = lambda t: f'?tags=${t}&format=json&jsoncallback=?'
url = lambda t: f'https://${host}${path}${query(t)}'

In [8]:
import requests
from IPython.display import display, HTML

getJSON = utils.compose(requests.models.Response.json, requests.get)
printHTML = utils.compose(display, HTML)

In [9]:
app = utils.compose(getJSON, utils.trace('response'), url)

In [11]:
utils.errorHandle(app)('cats')

response https://$api.flickr.com$/services/feeds/photos_public.gne$?tags=$cats&format=json&jsoncallback=?
Error: HTTPSConnectionPool(host='$api.flickr.com$', port=443): Max retries exceeded with url: /services/feeds/photos_public.gne$?tags=$cats&format=json&jsoncallback=? (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B23C999490>: Failed to resolve '$api.flickr.com$' ([Errno 11001] getaddrinfo failed)"))
